In [ ]:
#import geocoder
import pandas as pd
import re
import numpy as np
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Clean Location Variabe


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/NLP_Group 19/vaccination_all_tweets.csv")

In [ ]:
df.head(100)

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1338548332218630144,Evan Webeck,Oakland,S̶p̶o̶r̶t̶s̶ coronavirus reporter for @mercnew...,2013-12-02 20:20:02,1296,543,10844,True,2020-12-14 18:16:06,There is some good news: The first doses of th...,['PfizerBioNTech'],TweetDeck,0,1,False
96,1338546341404930050,Human_9.0,Madchester,Fighting For Freedom 🇬🇧 🇺🇸 🇨🇦 🇦🇺 🇳🇿\nI'm not a...,2020-10-07 20:38:10,590,987,7609,False,2020-12-14 18:08:11,WHERE IS THE PANDEMIC ? #PCRGATE #Condemic #Bi...,"['PCRGATE', 'Condemic', 'BillGatesIsNotADoctor...",Twitter Web App,0,2,False
97,1338545220691038211,Most Audacious Idol,NaN,NaN,2020-11-29 06:20:55,60,98,2076,False,2020-12-14 18:03:44,Make vaccine companies liable again... #Pfizer...,"['PfizerBioNTech', 'pfizer', 'johnsonandtoxin'...",Twitter Web App,0,0,False
98,1338544403795881987,The New Dispatch,NaN,News News and more News,2019-08-23 01:54:10,115,74,2,False,2020-12-14 18:00:29,The FDA Authorizes Emergency Use Of The Pfizer...,"['PFE', 'Pfizer', 'Pfizervaccine']",Twitter Web App,0,0,False


In [ ]:
def clean_text(txt_in):
    import re
    clean = re.sub('[^A-Za-z]+', " ", str(txt_in)).strip()
    return clean


In [ ]:
df['user_location']=df['user_location'].apply(clean_text)

In [ ]:
df.loc[df['user_name'] == 'Farooq Anees'] #success

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
37,1337734069816348673,Farooq Anees,,Somethings are better not be disclosed.... \o/,2010-12-23 11:51:01,293,319,165,False,2020-12-12 12:20:30,"Ladies and Gentlemen, now we have #CovidVaccin...","['CovidVaccine', 'Pakistan', 'PfizerBioNTech',...",Twitter for Android,0,0,False


In [ ]:
df_dup = pd.DataFrame()

In [ ]:
df_dup['location'] = df["user_location"].unique()

# This method maybe too slow to catch all the country.

In [ ]:
#
# from geopy.geocoders import Nominatim


# from geopy.extra.rate_limiter import RateLimiter
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# df_dup['2_location'] = df_dup['location'].apply(geocode)

# df_dup['point'] = df_dup['2_location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [ ]:
states = {
    "AL": "Alabama",
    "AK": "Alaska",
    "AS": "American Samoa",
    "AZ": "Arizona",
    "AR": "Arkansas",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "DC": "District Of Columbia",
    "FM": "Federated States Of Micronesia",
    "FL": "Florida",
    "GA": "Georgia",
    "GU": "Guam",
    "HI": "Hawaii",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "IA": "Iowa",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "ME": "Maine",
    "MH": "Marshall Islands",
    "MD": "Maryland",
    "MA": "Massachusetts",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MS": "Mississippi",
    "MO": "Missouri",
    "MT": "Montana",
    "NE": "Nebraska",
    "NV": "Nevada",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NY": "New York",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "MP": "Northern Mariana Islands",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PW": "Palau",
    "PA": "Pennsylvania",
    "PR": "Puerto Rico",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VT": "Vermont",
    "VI": "Virgin Islands",
    "VA": "Virginia",
    "WA": "Washington",
    "WV": "West Virginia",
    "WI": "Wisconsin",
    "WY": "Wyoming"
}

In [ ]:
states_df = pd.DataFrame(list(states.items()),columns = ['State_Code','State']) 

In [ ]:
country_names = pd.read_csv('/content/drive/MyDrive/NLP_Group 19/country_names.csv')

In [ ]:

def process_C_C(x):
    for i in states_df['State']:
        if i in x:
            return 'USA'
    for i in states_df['State_Code']:
        if i in x:
            return 'USA'            
    for i in country_names['Name']:
        if i in x:
            return i

            

df['Country'] = df['user_location'].apply(process_C_C)


In [ ]:
df = df[df['Country'].notnull()]

In [ ]:

df['Country'] = df['Country'].replace(['United States'],'USA')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,Country
0,1340539111971516416,Rachel Roh,La Crescenta Montrose CA,Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False,USA
1,1338158543359250433,Albert Fong,San Francisco CA,"Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False,USA
3,1337855739918835717,Charles Adler,Vancouver BC Canada,"Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129,False,Canada
6,1337851215875608579,Gunther Fehlinger,Austria Ukraine and Kosovo,End North Stream 2 now - the pipeline of corru...,2013-06-10 17:49:22,2731,5001,69344,False,2020-12-12 20:06:00,it is a bit sad to claim the fame for success ...,['vaccination'],Twitter Web App,0,4,False,Austria
10,1337841934170255365,Tamer Yazar,Turkey Israel,"Im Market Analyst, also Editor... working (fre...",2009-09-17 16:45:16,1302,78,339,False,2020-12-12 19:29:07,"while deaths are closing in on the 300,000 mar...","['PfizerBioNTech', 'Vaccine']",Twitter Web App,0,0,False,Israel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223282,1457218863737671680,VaxBLR,Bengaluru India,Hourly updates on FREE and PAID 18+ and 45+ va...,2021-06-21 08:44:34,31,0,0,False,2021-11-07 05:30:25,18-44 #BBMP #Bengaluru #CovidVaccine Availabil...,"['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHI...",VaxBlr,0,1,False,India
223283,1457203761676312585,VaxBLR,Bengaluru India,Hourly updates on FREE and PAID 18+ and 45+ va...,2021-06-21 08:44:34,31,0,0,False,2021-11-07 04:30:24,18-44 #BBMP #Bengaluru #CovidVaccine Availabil...,"['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHI...",VaxBlr,0,1,False,India
223284,1457188663230492674,VaxBLR,Bengaluru India,Hourly updates on FREE and PAID 18+ and 45+ va...,2021-06-21 08:44:34,31,0,0,False,2021-11-07 03:30:25,18-44 #BBMP #Bengaluru #CovidVaccine Availabil...,"['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHI...",VaxBlr,0,1,False,India
223285,1457173563215994883,VaxBLR,Bengaluru India,Hourly updates on FREE and PAID 18+ and 45+ va...,2021-06-21 08:44:34,31,0,0,False,2021-11-07 02:30:25,18-44 #BBMP #Bengaluru #CovidVaccine Availabil...,"['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHI...",VaxBlr,0,1,False,India


In [ ]:
freq_df = df.groupby(['Country']).size()

In [ ]:
freq_df = freq_df.to_frame()

In [ ]:
freq_df.columns = ['Freq']

In [ ]:
freq_df = freq_df.sort_values(['Freq'], ascending=False)

In [ ]:
freq_df.head(10)

,Freq
Country,
India,43302
USA,28419
Canada,5334
Sri Lanka,2058
Pakistan,2021
China,1513
Philippines,1169
United Kingdom,1160
Malaysia,1142


The Below codes are trying to convert user_location to country based on cities. However, it failed because of unknown bug. If you still want to try, you can do it.  So, far I filtered out users that have specific 'country' name in user location. The dataset is still big. - Lucas 

In [ ]:
# world_cities = pd.read_csv('/content/drive/MyDrive/NLP_Group 19/worldcities.csv')

In [ ]:
# res = [key for key, val in world_dict.items() if search_key in key]
# res

In [ ]:
# import geopy
# #!pip install geopandas
# import geopandas
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="nameh")

In [ ]:
#world_dict = dict(zip(world_cities.city, world_cities.country))

In [ ]:
#list_ = list(test_df['user_location'])

In [ ]:
# from geopy.extra.rate_limiter import RateLimiter

# # 1 - conveneint function to delay between geocoding calls
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# # 2- - create location column
# test_df['location_1'] = test_df['user_location'].apply(geocode)
# # 3 - create longitude, laatitude and altitude from location column (returns tuple)
# test_df['point'] = test_df['location_1'].apply(lambda loc: tuple(loc.point) if loc else None)
# # 4 - split point column into latitude, longitude and altitude columns
# test_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(test_df['point'].tolist(), index=df.index)

In [ ]:
# world_cities.loc[1,'country']

In [ ]:
# for i in world_cities['city']:
#   if i in list_:
#     print(i)
#   else: print('None')

In [ ]:
# def process_C_C2(x):
#     for i in world_cities['city']:
#         if i in x:
#             country = world_cities.loc[i,'country']
#             print(country)
#             return country


            

# test_df['Country1'] = test_df['user_location'].apply(process_C_C2)

In [ ]:
# list_have = []
# for x,y in world_dict.items():
#   if x in list_:
#     print(x)
#     list_have.append(y)
#   else: 
#     list_have.append(None)

In [ ]:
df.head(100)

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,Country
0,1340539111971516416,Rachel Roh,La Crescenta Montrose CA,Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False,USA
1,1338158543359250433,Albert Fong,San Francisco CA,"Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False,USA
3,1337855739918835717,Charles Adler,Vancouver BC Canada,"Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129,False,Canada
6,1337851215875608579,Gunther Fehlinger,Austria Ukraine and Kosovo,End North Stream 2 now - the pipeline of corru...,2013-06-10 17:49:22,2731,5001,69344,False,2020-12-12 20:06:00,it is a bit sad to claim the fame for success ...,['vaccination'],Twitter Web App,0,4,False,Austria
10,1337841934170255365,Tamer Yazar,Turkey Israel,"Im Market Analyst, also Editor... working (fre...",2009-09-17 16:45:16,1302,78,339,False,2020-12-12 19:29:07,"while deaths are closing in on the 300,000 mar...","['PfizerBioNTech', 'Vaccine']",Twitter Web App,0,0,False,Israel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,1338143123071971328,Realtalktimenews,United States,News. Entertainment. Viral. & Breaking.,2019-06-20 01:29:38,103,271,93,False,2020-12-13 15:25:56,The first trucks loaded with Pfizer’s coronavi...,NaN,Twitter Web App,0,0,False,USA
223,1338143061558390784,Lel B.,United States,#BidenHarris2020. Nobody calls our Military Lo...,2009-03-14 18:13:30,2762,3295,33886,False,2020-12-13 15:25:42,@JoeBiden @KamalaHarris Thanks to you both for...,NaN,Twitter for iPhone,0,1,False,USA
227,1338136603038949378,AFP News Agency,France,Top news and features from AFP's reporters aro...,2011-09-27 00:35:10,1953417,618,2338,True,2020-12-13 15:00:02,#UPDATE The #PfizerBioNTech Covid-19 vaccine b...,"['UPDATE', 'PfizerBioNTech']",TweetDeck,33,48,False,France
229,1338129045087637504,Times of Oman,Oman,Oman's leading English language media house. h...,2014-03-20 04:50:33,88187,496,825,True,2020-12-13 14:30:00,Rival US companies #UPS and #FedEx are working...,"['UPS', 'FedEx', 'PfizerBioNTech', 'Covid', 'v...",TweetDeck,0,5,False,Oman


In [ ]:
# for i in test_df['user_location']:
#   if i in world_cities['city']:
#     print(i)
# def process_C_C(x):
#     for i,y in world_cities['city']:
#         if i in x['user_location']:
#             x['Country1'] =  world_cities['country']
#             print(world_cities['country'])
#             return test_df
# process_C_C(test_df)
    #if i in test_df['user_location']:
      #print(i)
      #test_df['Country_1'] = world_cities['country']

In [ ]:
##Clean Text Column 潦草版

# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# import pandas as pd
# import re
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize


# def clean(string : str):
#     string = "".join(re.findall("[a-zA-Z]+\\s", string))
#     token = word_tokenize(string)
#     tokens_without_sw = [word for word in token if not word in stopwords.words('english')]
#     return " ".join(tokens_without_sw)

# def clear_text(file):

#     df['text'] = df['text'].apply(lambda x: clean(x))
#     return df

# if __name__ == '__main__':
#     df = clear_text('vaccination_all_tweets.csv')
#     print(df['text'])   

Remove punctuations   
Tokenization - Converting a sentence into list of words  
Remove stopwords  
Lammetization/stemming - Tranforming any form of a word to its root word  

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
import re
import string
from bs4 import BeautifulSoup
!pip install emoji

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 170 kB 27.0 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=5e483f281923f201ffe96676d00f4e20697a038281d8a42f0ef064c07086bbe2
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [ ]:
#df['text']=df['text'].astype(str).apply(lambda x: x.encode('latin-1', 'ignore').decode('latin-1'))

In [ ]:

df['text'] = df['text'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

In [ ]:
df['text']

0         Same folks said daikon paste could treat a cyt...
1         While the world has been on the wrong side of ...
3         Facts are immutable Senator even when youre no...
6         it is a bit sad to claim the fame for success ...
10        while deaths are closing in on the  mark milli...
                                ...                        
223282     BBMP Bengaluru CovidVaccine Availability for ...
223283     BBMP Bengaluru CovidVaccine Availability for ...
223284     BBMP Bengaluru CovidVaccine Availability for ...
223285     BBMP Bengaluru CovidVaccine Availability for ...
223286     BBMP Bengaluru CovidVaccine Availability for ...
Name: text, Length: 99936, dtype: object

In [ ]:
def mention(string):
  import re
  string = re.sub(r'@[A-Za-z0-9]+','',string)
  return string




In [ ]:
df['text'] = df['text'].apply(lambda x: mention(str(x)))

In [ ]:
def http(string:str):
  import re
  string = re.sub('https?://[A-Za-z0-9./]+','',string)
  return string

In [ ]:
df['text'] = df['text'].apply(lambda x: http(x))

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

In [ ]:
df['text'] = df['text'].apply(lambda x: remove_punct(x))

In [ ]:
df.loc[58,'text']

'Getting my first dose of the PfizerBioNTech CovidVaccine next weekend and seeing this UNBOXING just made it so r '

In [ ]:
# def remove_stopwords(text):
#     text = "".join(re.findall("[a-zA-Z]+\\s", string))
#     text = [word for word in text if word not in stopwords.words('english')]
#     return text
    
# df['text'] = df['text'].apply(lambda x: remove_stopwords(x))

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['text_no_sw'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
df['text_no_sw'] = df['text_no_sw'].str.lower()

In [ ]:
#remove emoji
df['text_no_sw']= df['text_no_sw'].astype(str).apply(lambda x: x.encode('latin-1', 'ignore').decode('latin-1'))


In [ ]:
from nltk.tokenize import word_tokenize
#df['token'] = df["text_no_sw"].apply(word_tokenize)
df['tokenized_text'] = df['text_no_sw'].apply(word_tokenize) 
df.head()


,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,Country,text_no_sw,tokenized_text,Tweet_stemmed,lemma
0,1340539111971516416,Rachel Roh,La Crescenta Montrose CA,Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False,USA,same folks said daikon paste could treat cytok...,"[same, folks, said, daikon, paste, could, trea...","[same, folk, said, daikon, past, could, treat,...","[same, folk, said, daikon, paste, could, treat..."
1,1338158543359250433,Albert Fong,San Francisco CA,"Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False,USA,while world wrong side history year hopefully ...,"[while, world, wrong, side, history, year, hop...","[while, world, wrong, side, histori, year, hop...","[while, world, wrong, side, history, year, hop..."
3,1337855739918835717,Charles Adler,Vancouver BC Canada,"Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,Facts are immutable Senator even when youre no...,NaN,Twitter Web App,446,2129,False,Canada,facts immutable senator even youre ethically s...,"[facts, immutable, senator, even, youre, ethic...","[fact, immut, senat, even, your, ethic, sturdi...","[fact, immutable, senator, even, youre, ethica..."
6,1337851215875608579,Gunther Fehlinger,Austria Ukraine and Kosovo,End North Stream 2 now - the pipeline of corru...,2013-06-10 17:49:22,2731,5001,69344,False,2020-12-12 20:06:00,it is a bit sad to claim the fame for success ...,['vaccination'],Twitter Web App,0,4,False,Austria,bit sad claim fame success vaccination patriot...,"[bit, sad, claim, fame, success, vaccination, ...","[bit, sad, claim, fame, success, vaccin, patri...","[bit, sad, claim, fame, success, vaccination, ..."
10,1337841934170255365,Tamer Yazar,Turkey Israel,"Im Market Analyst, also Editor... working (fre...",2009-09-17 16:45:16,1302,78,339,False,2020-12-12 19:29:07,while deaths are closing in on the mark milli...,"['PfizerBioNTech', 'Vaccine']",Twitter Web App,0,0,False,Israel,deaths closing mark millions people wait pfize...,"[deaths, closing, mark, millions, people, wait...","[death, close, mark, million, peopl, wait, pfi...","[death, closing, mark, million, people, wait, ..."


In [ ]:
stem = nltk.PorterStemmer()
def stemming(text):
    text = [stem.stem(word) for word in text]
    return text

df['Tweet_stemmed'] = df['tokenized_text'].apply(lambda x: stemming(x))
df.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,Country,text_no_sw,tokenized_text,Tweet_stemmed,lemma
0,1340539111971516416,Rachel Roh,La Crescenta Montrose CA,Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False,USA,same folks said daikon paste could treat cytok...,"[same, folks, said, daikon, paste, could, trea...","[same, folk, said, daikon, past, could, treat,...","[same, folk, said, daikon, paste, could, treat..."
1,1338158543359250433,Albert Fong,San Francisco CA,"Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False,USA,while world wrong side history year hopefully ...,"[while, world, wrong, side, history, year, hop...","[while, world, wrong, side, histori, year, hop...","[while, world, wrong, side, history, year, hop..."
3,1337855739918835717,Charles Adler,Vancouver BC Canada,"Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,Facts are immutable Senator even when youre no...,NaN,Twitter Web App,446,2129,False,Canada,facts immutable senator even youre ethically s...,"[facts, immutable, senator, even, youre, ethic...","[fact, immut, senat, even, your, ethic, sturdi...","[fact, immutable, senator, even, youre, ethica..."
6,1337851215875608579,Gunther Fehlinger,Austria Ukraine and Kosovo,End North Stream 2 now - the pipeline of corru...,2013-06-10 17:49:22,2731,5001,69344,False,2020-12-12 20:06:00,it is a bit sad to claim the fame for success ...,['vaccination'],Twitter Web App,0,4,False,Austria,bit sad claim fame success vaccination patriot...,"[bit, sad, claim, fame, success, vaccination, ...","[bit, sad, claim, fame, success, vaccin, patri...","[bit, sad, claim, fame, success, vaccination, ..."
10,1337841934170255365,Tamer Yazar,Turkey Israel,"Im Market Analyst, also Editor... working (fre...",2009-09-17 16:45:16,1302,78,339,False,2020-12-12 19:29:07,while deaths are closing in on the mark milli...,"['PfizerBioNTech', 'Vaccine']",Twitter Web App,0,0,False,Israel,deaths closing mark millions people wait pfize...,"[deaths, closing, mark, millions, people, wait...","[death, close, mark, million, peopl, wait, pfi...","[death, closing, mark, million, people, wait, ..."


In [ ]:
nltk.download('wordnet')
lemma = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [lemma.lemmatize(word) for word in text]
    return text

df['lemma'] = df['tokenized_text'].apply(lambda x: lemmatizer(x))
df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,Country,text_no_sw,tokenized_text,Tweet_stemmed,lemma
0,1340539111971516416,Rachel Roh,La Crescenta Montrose CA,Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False,USA,same folks said daikon paste could treat cytok...,"[same, folks, said, daikon, paste, could, trea...","[same, folk, said, daikon, past, could, treat,...","[same, folk, said, daikon, paste, could, treat..."
1,1338158543359250433,Albert Fong,San Francisco CA,"Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False,USA,while world wrong side history year hopefully ...,"[while, world, wrong, side, history, year, hop...","[while, world, wrong, side, histori, year, hop...","[while, world, wrong, side, history, year, hop..."
3,1337855739918835717,Charles Adler,Vancouver BC Canada,"Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,Facts are immutable Senator even when youre no...,NaN,Twitter Web App,446,2129,False,Canada,facts immutable senator even youre ethically s...,"[facts, immutable, senator, even, youre, ethic...","[fact, immut, senat, even, your, ethic, sturdi...","[fact, immutable, senator, even, youre, ethica..."
6,1337851215875608579,Gunther Fehlinger,Austria Ukraine and Kosovo,End North Stream 2 now - the pipeline of corru...,2013-06-10 17:49:22,2731,5001,69344,False,2020-12-12 20:06:00,it is a bit sad to claim the fame for success ...,['vaccination'],Twitter Web App,0,4,False,Austria,bit sad claim fame success vaccination patriot...,"[bit, sad, claim, fame, success, vaccination, ...","[bit, sad, claim, fame, success, vaccin, patri...","[bit, sad, claim, fame, success, vaccination, ..."
10,1337841934170255365,Tamer Yazar,Turkey Israel,"Im Market Analyst, also Editor... working (fre...",2009-09-17 16:45:16,1302,78,339,False,2020-12-12 19:29:07,while deaths are closing in on the mark milli...,"['PfizerBioNTech', 'Vaccine']",Twitter Web App,0,0,False,Israel,deaths closing mark millions people wait pfize...,"[deaths, closing, mark, millions, people, wait...","[death, close, mark, million, peopl, wait, pfi...","[death, closing, mark, million, people, wait, ..."


In [ ]:
df['text_no_sw'].head(25)

0     same folks said daikon paste could treat cytok...
1     while world wrong side history year hopefully ...
3     facts immutable senator even youre ethically s...
6     bit sad claim fame success vaccination patriot...
10    deaths closing mark millions people wait pfize...
12    the agency also released new information healt...
19    the trump administration failed deliver vaccin...
21    anyone wondering day pfizerbiontech approval u...
23    the us food drug administration fda granted em...
26    wear mask wash hands remain socially distant p...
27    pfizerbiontech fda approval apprentice style a...
28    interesting detailed article showing well test...
31    please dont spread fake news there evidence ca...
32    icymi the fda authorized pfizerbiontech covid ...
33    all respect due process give approval instead ...
39                      check table tables took penny w
40               one doesnt side effects pfizerbiontech
44    fda authorizes pfizerbiontech coronavirus 

In [ ]:
df.to_csv("/content/drive/MyDrive/NLP_Group 19/clean_csv.csv")